In [1]:
import os
os.chdir('/media/jorge/HDD/TFG/')
import json
import rosbag
import cv2 as cv
import numpy as np
import pyrealsense2 as rs

from cv_bridge import CvBridge
from pycocotools.coco import COCO
from skimage.feature import hog
from time import time

from src.depth2hha.getHHA import getHHA
from src.utils.postprocessing import depth_to_normal

BAGFILE_PATH = 'data/data_base/lindenthal-camera-traps/bagfiles'
DATASET_PATH = 'data/lindenthal-camera-traps/lindenthal_coco/'
OUTPUT_PATH = 'data/lindenthal-camera-traps/lindenthal_coco/images'

In [ ]:
import os
os.chdir('/media/jorge/HDD/TFG/')
import json
import rosbag
import cv2 as cv
import numpy as np
import pyrealsense2 as rs
import matplotlib.pyplot as plt

from cv_bridge import CvBridge
from pycocotools.coco import COCO
from skimage.feature import hog
from time import time

from src.depth2hha.getHHA import getHHA
from src.utils.postprocessing import depth_to_normal, equalize_histogram

BAGFILE_PATH = 'data/data_base/lindenthal-camera-traps/bagfiles'
DATASET_PATH = 'data/lindenthal-camera-traps/lindenthal_coco/'
OUTPUT_PATH = 'data/lindenthal-camera-traps/lindenthal_coco/images'


# iterate over all the videos inside the output path
for video_folder in os.listdir(OUTPUT_PATH):
    for frame_fn in os.listdir(os.path.join(OUTPUT_PATH, video_folder, 'depth')):
        # only process the multiples of 10 plus 1
        if (int(frame_fn.split('.')[0]) - 1) % 10 != 0:
            continue

        if int(frame_fn.split('.')[0]) < 40:
            continue
        
        # read the frame
        frame = cv.imread(os.path.join(OUTPUT_PATH, video_folder, 'depth', frame_fn), cv.IMREAD_UNCHANGED)
        
        # plot the histogram of values of the image
        depth_values = frame.flatten()

        # Calculate histogram (ignoring zero values, if desired)
        histogram, bin_edges = np.histogram(depth_values[depth_values > 0], bins=range(0, 65536), density=False)

        # Plot the histogram
        plt.figure(figsize=(10, 6))
        plt.bar(bin_edges[:-1], histogram, width=1, color='blue', alpha=0.7, edgecolor='black')
        plt.title('Depth Image Histogram')
        plt.xlabel('Depth Value (millimeters)')
        plt.ylabel('Frequency')
        plt.xlim([1, 65535])  # Ignore zero values for better visualization
        plt.yscale('log')  # Use a logarithmic scale for better visualization of distributions
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.show()

        # equalize the histogram of the image
        eq_frame = equalize_histogram(frame)
        eq_depth_values = eq_frame.flatten()
        eq_histogram, eq_bin_edges = np.histogram(eq_depth_values[eq_depth_values > 0], bins=range(0, 65536), density=False)
        plt.figure(figsize=(10, 6))
        plt.bar(eq_bin_edges[:-1], eq_histogram, width=1, color='blue', alpha=0.7, edgecolor='black')
        plt.title('Equalized Depth Image Histogram')
        plt.xlabel('Depth Value (arbitrary units)')
        plt.ylabel('Frequency')
        plt.xlim([1, 65535])
        plt.yscale('log')
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.show()
        break
    break

In [8]:
import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
os.chdir('/media/jorge/HDD/TFG/')
import json
import rosbag
import cv2 as cv
import numpy as np
import pyrealsense2 as rs

from cv_bridge import CvBridge
from pycocotools.coco import COCO
from skimage.feature import hog
from time import time

from src.depth2hha.getHHA import getHHA
from src.utils.postprocessing import depth_to_normal

DATASET_PATH = 'data/lindenthal-camera-traps/lindenthal_coco/'
OUTPUT_PATH = 'data/lindenthal-camera-traps/lindenthal_coco/images'

# iterate over all the videos inside the output path
for video_folder in os.listdir(OUTPUT_PATH):
    for frame_fn in os.listdir(os.path.join(OUTPUT_PATH, video_folder, 'depth')):
        # only process the multiples of 10 plus 1
        # if (int(frame_fn.split('.')[0]) - 1) % 10 != 0:
        #     continue

        # if int(frame_fn.split('.')[0]) < 40:
        #     continue
        
        # read the frame
        frame = cv.imread(os.path.join(OUTPUT_PATH, video_folder, 'depth', frame_fn), cv.IMREAD_ANYCOLOR | cv.IMREAD_UNCHANGED)
        
        # generate the hha image and save it to the 'hha' folder
        with open(os.path.join(OUTPUT_PATH, video_folder, 'intrinsics.json'), 'r') as intrinsics_file:
            intrinsics = json.load(intrinsics_file)
        
        cam_matrix = np.matrix([
            [intrinsics['fx'],  0,                  intrinsics['ppx']],
            [0,                 intrinsics['fy'],   intrinsics['ppy']],
            [0,                 0,                  1]
        ])
        # apply the HHA transformation
        hha_map = getHHA(cam_matrix, frame)
        # save the image
        # create the output folder if it does not exist
        if not os.path.exists(os.path.join(OUTPUT_PATH, video_folder, 'hha')):
            os.makedirs(os.path.join(OUTPUT_PATH, video_folder, 'hha'))
        cv.imwrite(os.path.join(OUTPUT_PATH, video_folder, 'hha', frame_fn.split('.')[0]+'.png'), hha_map)
        print(f'Image saved to {os.path.join(OUTPUT_PATH, video_folder, "hha", frame_fn)}')

Image saved to data/lindenthal-camera-traps/lindenthal_coco/images/20200925072315/hha/000041.exr
Image saved to data/lindenthal-camera-traps/lindenthal_coco/images/20200925072315/hha/000281.exr
Image saved to data/lindenthal-camera-traps/lindenthal_coco/images/20200925072315/hha/000211.exr
Image saved to data/lindenthal-camera-traps/lindenthal_coco/images/20200925072315/hha/000061.exr
Image saved to data/lindenthal-camera-traps/lindenthal_coco/images/20200925072315/hha/000361.exr
Image saved to data/lindenthal-camera-traps/lindenthal_coco/images/20200925072315/hha/000411.exr
Image saved to data/lindenthal-camera-traps/lindenthal_coco/images/20200925072315/hha/000171.exr
Image saved to data/lindenthal-camera-traps/lindenthal_coco/images/20200925072315/hha/000011.exr
Image saved to data/lindenthal-camera-traps/lindenthal_coco/images/20200925072315/hha/000301.exr
Image saved to data/lindenthal-camera-traps/lindenthal_coco/images/20200925072315/hha/000311.exr
Image saved to data/lindenthal

In [ ]:
import os
os.chdir('/media/jorge/HDD/TFG/')
import json
import rosbag
import cv2 as cv
import numpy as np
import pyrealsense2 as rs
import matplotlib.pyplot as plt

from cv_bridge import CvBridge
from pycocotools.coco import COCO
from skimage.feature import hog
from time import time

from src.depth2hha.getHHA import getHHA
from src.utils.postprocessing import depth_to_normal, depth_to_colormap, equalize_histogram

BAGFILE_PATH = 'data/data_base/lindenthal-camera-traps/bagfiles'
DATASET_PATH = 'data/lindenthal-camera-traps/lindenthal_coco/'
OUTPUT_PATH = 'data/lindenthal-camera-traps/lindenthal_coco/images'

# iterate over all the videos inside the output path
for video_folder in os.listdir(OUTPUT_PATH):
    for frame_fn in os.listdir(os.path.join(OUTPUT_PATH, video_folder, 'depth')):
        # only process the multiples of 10 plus 1
        # if (int(frame_fn.split('.')[0]) - 1) % 10 != 0:
        #     continue

        # if int(frame_fn.split('.')[0]) < 40:
        #     continue
        
        # only process the frame if video is 20200807061001 and frame is 000031
        if video_folder != '20200807061001' or frame_fn != '000031.png':
            continue
        # read the frame
        frame = cv.imread(os.path.join(OUTPUT_PATH, video_folder, 'depth', frame_fn), cv.IMREAD_UNCHANGED)
        start = time()
        eq_frame = equalize_histogram(frame)
        jet_map = depth_to_colormap(frame, colormap='jet', equalize=True)
        normal_map = depth_to_normal(frame, equalize=True)
        # read the intrinsics
        with open(os.path.join(OUTPUT_PATH, video_folder, 'intrinsics.json'), 'r') as intrinsics_file:
            intrinsics = json.load(intrinsics_file)
        cam_matrix = np.matrix([
            [intrinsics['fx'],  0,                  intrinsics['ppx']],
            [0,                 intrinsics['fy'],   intrinsics['ppy']],
            [0,                 0,                  1]
        ])
        # apply the HHA transformation
        hha_map = getHHA(cam_matrix, frame)
        hha_map_eq = getHHA(cam_matrix, eq_frame)
        distance_map = depth_to_colormap(frame, colormap='distance', equalize=True)
        print(f'Processing time {time()-start} ms')


        cv.imshow('depth', frame)
        cv.imshow('equalized', eq_frame)
        cv.imshow('jet', jet_map)
        cv.imshow('distance', distance_map)
        cv.imshow('normals', normal_map)
        cv.imshow('HHA', hha_map)
        cv.imshow('HHA_eq', hha_map_eq)
        cv.waitKey(0)
        break

Processing time 8.455435037612915 ms


In [ ]:
import os
os.chdir('/media/jorge/HDD/TFG/')
import json
import rosbag
import cv2 as cv
import numpy as np
import pyrealsense2 as rs
import matplotlib.pyplot as plt

from cv_bridge import CvBridge
from pycocotools.coco import COCO
from skimage.feature import hog
from time import time

from src.depth2hha.getHHA import getHHA
from src.utils.postprocessing import depth_to_normal, depth_to_jet

BAGFILE_PATH = 'data/data_base/lindenthal-camera-traps/bagfiles'
DATASET_PATH = 'data/lindenthal-camera-traps/lindenthal_coco/'
OUTPUT_PATH = 'data/lindenthal-camera-traps/lindenthal_coco/images'

# iterate over all the videos inside the output path
for video_folder in os.listdir(OUTPUT_PATH):
    for frame_fn in os.listdir(os.path.join(OUTPUT_PATH, video_folder, 'depth')):
        # only process the multiples of 10 plus 1
        # if (int(frame_fn.split('.')[0]) - 1) % 10 != 0:
        #     continue

        # if int(frame_fn.split('.')[0]) < 40:
        #     continue
        
        # read the frame
        frame = cv.imread(os.path.join(OUTPUT_PATH, video_folder, 'depth', frame_fn), cv.IMREAD_UNCHANGED)    

        start = time()
        # apply RGB colormap (map depth to rgb colors where near is red and far is blue)
        hist, bins = np.histogram(frame.flatten(), bins=65536, range=[1, 65535])
        # Calculate the cumulative distribution function (CDF)
        cdf = hist.cumsum()
        cdf_normalized = cdf / cdf[-1]
        # Use the CDF to map the original image values to the equalized values
        equalized = np.interp(frame.flatten(), bins[:-1], cdf_normalized * 65535)
        # Reshape back to the original image shape and cast to uint16
        equalized_histogram = equalized.reshape(frame.shape).astype(np.uint16)


        # apply a jet colormap to the depth image
        # Normalize the uint16 image to the range [0, 1]
        # Normalize the uint16 image to the range [0, 1]
        normalized_image = equalized_histogram.astype(np.float64) / 65535.0

        # Define the JET colormap manually in float64 precision
        jet_colormap = np.array([
            [0.5, 0, 0],     # Dark red
            [1, 0, 0],       # Red
            [1, 0.5, 0],     # Orange
            [1, 1, 0],       # Yellow
            [0.5, 1, 0.5],   # Green
            [0, 1, 1],       # Light cyan
            [0, 0.5, 1],     # Cyan
            [0, 0, 1],       # Blue
            [0, 0, 0.5],     # Dark blue
        ], dtype=np.float64)

        # Generate interpolated colormap values
        indices = np.linspace(0, 1, len(jet_colormap))
        red_interp = np.interp(normalized_image.flat, indices, jet_colormap[:, 0]).reshape(frame.shape)
        green_interp = np.interp(normalized_image.flat, indices, jet_colormap[:, 1]).reshape(frame.shape)
        blue_interp = np.interp(normalized_image.flat, indices, jet_colormap[:, 2]).reshape(frame.shape)

        # Combine channels and scale back to uint16 range [0, 65535]
        colormapped_image = np.stack([
            (red_interp * 65535).astype(np.uint16),
            (green_interp * 65535).astype(np.uint16),
            (blue_interp * 65535).astype(np.uint16)
        ], axis=-1)

        print(f'Processing time {time()-start} ms')

        cv.imshow('depth', equalized_histogram)
        cv.imshow('rgb_map', colormapped_image)
        cv.imshow('jet_func', depth_to_jet(frame, equalize=True))
        cv.waitKey(0)
        break
        

Processing time 0.026830196380615234 ms


In [ ]:
import os
os.chdir('/media/jorge/HDD/TFG/')
import json
import rosbag
import cv2 as cv
import numpy as np
import pyrealsense2 as rs

from cv_bridge import CvBridge
from pycocotools.coco import COCO
from skimage.feature import hog
from time import time

from src.depth2hha.getHHA import getHHA
from src.utils.postprocessing import depth_to_normal

BAGFILE_PATH = 'data/data_base/lindenthal-camera-traps/bagfiles'
DATASET_PATH = 'data/lindenthal-camera-traps/lindenthal_coco/'
OUTPUT_PATH = 'data/lindenthal-camera-traps/lindenthal_coco/images'


# load the coco dataset
split = 'train'
coco = COCO(os.path.join(DATASET_PATH, f'{split}.json'))

# iterate over all the images in the coco dataset
for img_id in coco.getImgIds():
    video_id, frame_id = img_id.split('-')
    frame_id = frame_id.zfill(6) + '.png'

    # read the frame
    depth_frame = cv.imread(os.path.join(OUTPUT_PATH, video_id, 'depth', frame_id), cv.IMREAD_UNCHANGED)

    # read the intrinsics
    with open(os.path.join(OUTPUT_PATH, video_id, 'intrinsics.json'), 'r') as intrinsics_file:
        intrinsics = json.load(intrinsics_file)
    cam_matrix = np.matrix([
        [intrinsics['fx'],  0,                  intrinsics['ppx']],
        [0,                 intrinsics['fy'],   intrinsics['ppy']],
        [0,                 0,                  1]
    ])

    # apply the HHA transformation
    hha_frame = getHHA(cam_matrix, depth_frame)

    # save to the hha folder (create it if it doesn't exist)
    hha_folder = os.path.join(OUTPUT_PATH, video_id, 'hha')
    if not os.path.exists(hha_folder):
        os.makedirs(hha_folder)
    cv.imwrite(os.path.join(hha_folder, frame_id), hha_frame)
    
    print(f'Saved {os.path.join(hha_folder, frame_id)}')
    cv.waitKey(0)

loading annotations into memory...
Done (t=0.10s)
creating index...
index created!
Saved data/lindenthal-camera-traps/lindenthal_coco/images/20200807023726/hha/000031.png
Saved data/lindenthal-camera-traps/lindenthal_coco/images/20200807023726/hha/000041.png
Saved data/lindenthal-camera-traps/lindenthal_coco/images/20200807023726/hha/000051.png
Saved data/lindenthal-camera-traps/lindenthal_coco/images/20200807023726/hha/000061.png
Saved data/lindenthal-camera-traps/lindenthal_coco/images/20200807023726/hha/000071.png
Saved data/lindenthal-camera-traps/lindenthal_coco/images/20200807023726/hha/000081.png
Saved data/lindenthal-camera-traps/lindenthal_coco/images/20200807023726/hha/000091.png
Saved data/lindenthal-camera-traps/lindenthal_coco/images/20200807023726/hha/000111.png
Saved data/lindenthal-camera-traps/lindenthal_coco/images/20200807023726/hha/000121.png
Saved data/lindenthal-camera-traps/lindenthal_coco/images/20200807023726/hha/000131.png
Saved data/lindenthal-camera-traps/li

In [ ]:
import os
os.chdir('/media/jorge/HDD/TFG/')
import json
import rosbag
import cv2 as cv
import numpy as np
import pyrealsense2 as rs

from cv_bridge import CvBridge
from pycocotools.coco import COCO
from skimage.feature import hog
from time import time

from src.depth2hha.getHHA import getHHA
from src.utils.postprocessing import depth_to_normal

BAGFILE_PATH = 'data/data_base/lindenthal-camera-traps/bagfiles'
DATASET_PATH = 'data/lindenthal-camera-traps/lindenthal_coco/'
OUTPUT_PATH = 'data/lindenthal-camera-traps/lindenthal_coco/images'


# load the coco dataset
split = 'val'
coco = COCO(os.path.join(DATASET_PATH, f'{split}.json'))

# iterate over all the images in the coco dataset
for img_id in coco.getImgIds():
    video_id, frame_id = img_id.split('-')
    frame_id = frame_id.zfill(6) + '.png'

    # read the frame
    depth_frame = cv.imread(os.path.join(OUTPUT_PATH, video_id, 'depth', frame_id), cv.IMREAD_UNCHANGED)

    # read the intrinsics
    with open(os.path.join(OUTPUT_PATH, video_id, 'intrinsics.json'), 'r') as intrinsics_file:
        intrinsics = json.load(intrinsics_file)
    cam_matrix = np.matrix([
        [intrinsics['fx'],  0,                  intrinsics['ppx']],
        [0,                 intrinsics['fy'],   intrinsics['ppy']],
        [0,                 0,                  1]
    ])

    # apply the HHA transformation
    hha_frame = getHHA(cam_matrix, depth_frame)

    # save to the hha folder (create it if it doesn't exist)
    hha_folder = os.path.join(OUTPUT_PATH, video_id, 'hha')
    if not os.path.exists(hha_folder):
        os.makedirs(hha_folder)
    cv.imwrite(os.path.join(hha_folder, frame_id), hha_frame)
    
    print(f'Saved {os.path.join(hha_folder, frame_id)}')
    cv.waitKey(0)

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
Saved data/lindenthal-camera-traps/lindenthal_coco/images/20200807023726/hha/000101.png
Saved data/lindenthal-camera-traps/lindenthal_coco/images/20200807023726/hha/000141.png
Saved data/lindenthal-camera-traps/lindenthal_coco/images/20200807023726/hha/000361.png
Saved data/lindenthal-camera-traps/lindenthal_coco/images/20200807031150/hha/000071.png
Saved data/lindenthal-camera-traps/lindenthal_coco/images/20200807061001/hha/000041.png
Saved data/lindenthal-camera-traps/lindenthal_coco/images/20200807061001/hha/000141.png
Saved data/lindenthal-camera-traps/lindenthal_coco/images/20200807061001/hha/000151.png
Saved data/lindenthal-camera-traps/lindenthal_coco/images/20200807061001/hha/000171.png
Saved data/lindenthal-camera-traps/lindenthal_coco/images/20200807061001/hha/000331.png
Saved data/lindenthal-camera-traps/lindenthal_coco/images/20200807061001/hha/000361.png
Saved data/lindenthal-camera-traps/li

In [3]:
import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
os.chdir('/media/jorge/HDD/TFG/')
import json
import rosbag
import cv2 as cv
import numpy as np
import pyrealsense2 as rs
import matplotlib.pyplot as plt

from cv_bridge import CvBridge
from pycocotools.coco import COCO
from skimage.feature import hog
from time import time

from src.depth2hha.getHHA import getHHA
#from src.utils.postprocessing import depth_to_normal, depth_to_jet, equalize_histogram

BAGFILE_PATH = 'data/data_base/lindenthal-camera-traps/bagfiles'
DATASET_PATH = 'data/lindenthal-camera-traps/lindenthal_coco/'
OUTPUT_PATH = 'data/lindenthal-camera-traps/lindenthal_coco/images'

# iterate over all the videos inside the output path
mean_times = []
for video_folder in os.listdir(OUTPUT_PATH):
    video_times = []
    for frame_fn in os.listdir(os.path.join(OUTPUT_PATH, video_folder, 'depth')):
        # read the frame
        frame = cv.imread(os.path.join(OUTPUT_PATH, video_folder, 'depth', frame_fn), cv.IMREAD_UNCHANGED)
        start = time()
        #equalized_frame = equalize_histogram(frame)
        #jet_map = depth_to_jet(equalized_frame)
        with open(os.path.join(OUTPUT_PATH, video_folder, 'intrinsics.json'), 'r') as intrinsics_file:
            intrinsics = json.load(intrinsics_file)
        cam_matrix = np.matrix([
            [intrinsics['fx'],  0,                  intrinsics['ppx']],
            [0,                 intrinsics['fy'],   intrinsics['ppy']],
            [0,                 0,                  1]
        ])
        hha_frame = getHHA(cam_matrix, frame)
        frame_time = time()-start
        print(f'Processing time {frame_time} s')
        video_times.append(frame_time)
    mean_time = np.mean(video_times)
    print(f'Video {video_folder} mean processing time: {mean_time} s')
    mean_times.append(mean_time)
print(f'Mean processing time: {np.mean(mean_times)} s')

Processing time 4.287592887878418 s
Processing time 4.299122333526611 s
Processing time 4.33094334602356 s
Processing time 4.326228141784668 s
Processing time 4.136441707611084 s
Processing time 4.16982626914978 s
Processing time 4.2377400398254395 s
Processing time 4.618476152420044 s
Processing time 4.563860177993774 s
Processing time 4.218754291534424 s
Processing time 4.454778671264648 s
Processing time 4.31990385055542 s
Processing time 4.4097959995269775 s
Processing time 6.338006019592285 s
Processing time 4.649692058563232 s
Processing time 5.162703275680542 s
Processing time 4.460562229156494 s
Processing time 4.637681007385254 s
Processing time 4.324451446533203 s
Processing time 4.511049270629883 s
Processing time 4.324906349182129 s
Processing time 4.385570287704468 s
Processing time 4.371660232543945 s
Processing time 4.269977331161499 s
Processing time 4.307737112045288 s
Processing time 4.374964475631714 s
Processing time 4.49914026260376 s
Processing time 4.437056064605

KeyboardInterrupt: 